In [2]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 2.5 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
#import seaborn as sns
#import statsmodels.api as sm
from catboost import CatBoostClassifier, Pool
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

In [4]:
from tensorflow.keras.utils import to_categorical

In [126]:
alert_data = pd.read_csv("alert_output.csv")
alert_data.head()

,aircraft_id,engine_position,flight_phase,flight_datetime,alert_code,parameter_name,parameter_value,comparison_value,upper_limit,lower_limit,status,resolution,gen_datetime,time_to_peak,pilot_note,alerting_value,constituent_flag,cancel_flag,engine_id
0,9e6647a763e39e1f7882d046d44909f291eb4ae660ca2c...,2,CRUISE,2022-07-15 21:10:25,GE_10012,SLOATL,41.769714,46.783451,NaN,-3.0,OPEN,NaN,2022-07-15 09:36:20.000000,NaN,NaN,-5.013737,YES,NO,ebfab3e1c5709cd1c128c5549c28885a6aff8edf3d6af0...
1,a790052f10e5082a569e6e63a2378ab84defe36ff26c05...,1,TAKEOFF,2022-04-22 13:44:38,GE_8880,NaN,-999.000000,NaN,600.0,390.0,OPEN,NaN,2022-04-28 15:53:03.000000,NaN,NaN,NaN,NO,NO,1627d385f689d2145daa7c859b7cb6095e7dac0420a40c...
2,763a6ba60ec1b0ee633dda480ed9a9c285cd65a2f46372...,2,TAKEOFF,2022-01-28 23:18:55,GE_8880,NaN,-999.000000,NaN,600.0,390.0,OPEN,NaN,2022-04-28 15:53:03.000000,NaN,NaN,NaN,NO,NO,8731cb5c4b82a20e9d10305dd82da733bde060eecc0c77...
3,86b6b690d8ede50869aa2562b655a65abd786d169572b4...,2,TAKEOFF,2022-01-11 06:09:00,GE_8101,NaN,NaN,NaN,NaN,NaN,OPEN,NaN,2022-04-26 09:38:49.000000,NaN,NaN,NaN,NO,NO,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...
4,939c9231d25e2e31a7fef4ee60c025fb8d4bf7a15a904d...,1,CRUISE,2022-04-10 18:27:44,GE_10009,SLOATL,26.825850,NaN,NaN,33.0,OPEN,NaN,2022-05-04 12:30:40.000000,NaN,NaN,26.825850,YES,NO,8ab87b4892e1cafa720c1d6eb24de533a8bf69924bfe9e...


In [127]:
alert_data = alert_data.dropna(subset=['parameter_name', 'parameter_value'])

In [129]:
alert_data.head()

,aircraft_id,engine_position,flight_phase,flight_datetime,alert_code,parameter_name,parameter_value,comparison_value,upper_limit,lower_limit,status,resolution,gen_datetime,time_to_peak,pilot_note,alerting_value,constituent_flag,cancel_flag,engine_id
0,9e6647a763e39e1f7882d046d44909f291eb4ae660ca2c...,2,CRUISE,2022-07-15 21:10:25,GE_10012,SLOATL,41.769714,46.783451,NaN,-3.0,OPEN,NaN,2022-07-15 09:36:20.000000,NaN,NaN,-5.013737,YES,NO,ebfab3e1c5709cd1c128c5549c28885a6aff8edf3d6af0...
4,939c9231d25e2e31a7fef4ee60c025fb8d4bf7a15a904d...,1,CRUISE,2022-04-10 18:27:44,GE_10009,SLOATL,26.825850,NaN,NaN,33.0,OPEN,NaN,2022-05-04 12:30:40.000000,NaN,NaN,26.825850,YES,NO,8ab87b4892e1cafa720c1d6eb24de533a8bf69924bfe9e...
6,d08fc8f6785e23e13817d8a8aa17f927c1f5143a4202e3...,1,TAKEOFF,2022-04-12 11:26:22,GE_10023,EGTHDM,-11.224254,NaN,NaN,6.0,OPEN,NaN,2022-04-28 16:07:57.000000,NaN,NaN,-11.224254,YES,NO,83dd5ba4ee2f515b83c6f09ffd7268f0a389b6303f0bf3...
7,9e6647a763e39e1f7882d046d44909f291eb4ae660ca2c...,1,CRUISE,2022-04-27 07:13:28,GE_10005,DEGT,28.831909,15.129965,11.0,NaN,OPEN,NaN,2022-04-28 16:51:16.000000,NaN,NaN,13.701944,NO,NO,5f7a3f8e38df1ec0f02364d0b5a464a3ea6d0aa87f7599...
14,d08fc8f6785e23e13817d8a8aa17f927c1f5143a4202e3...,1,TAKEOFF,2022-04-10 10:13:02,GE_10023,EGTHDM,-9.157892,NaN,NaN,6.0,OPEN,NaN,2022-04-28 16:07:57.000000,NaN,NaN,-9.157892,YES,NO,83dd5ba4ee2f515b83c6f09ffd7268f0a389b6303f0bf3...


In [142]:
train_features = alert_data[["parameter_name", "parameter_value"]]
train_features = train_features.to_numpy()

In [143]:
train_features

array([['SLOATL', 41.7697144],
       ['SLOATL', 26.8258495],
       ['EGTHDM', -11.2242537],
       ...,
       ['DPOIL', -25.5710144],
       ['DPOIL', -25.2695618],
       ['DPOIL', -25.7600021]], dtype=object)

In [131]:
cat_features = [0]

In [132]:
train_labels = alert_data["alert_code"].to_numpy()
train_labels

array(['GE_10012', 'GE_10009', 'GE_10023', ..., 'GE_10019', 'GE_10019',
       'GE_10019'], dtype=object)

In [133]:
train_dataset = Pool(data=train_features,
                     label=train_labels,
                     cat_features=cat_features)

In [149]:
model = CatBoostClassifier(iterations=100,
                           learning_rate=0.5,
                           depth=2,
                           loss_function='MultiClass')

In [ ]:
model.fit(train_dataset)

In [156]:
preds_class = model.predict(['DPOIL', -245])
preds_class

array(['GE_10019'], dtype=object)